# Use case introduction 📰

In this notebook, we'll explore the use case of extracting structured information from news articles.

In short, given a news article, the goal is to extract the following information:
- The title of the article
- A summary of the article
- Whether the article is about business
- For each mentioned business
  - The name of the business
  - The likely stock price change (increase, decrease, none)
  - Reason for the stock price change
  - Relevant substring supporting the reason


## Exploring the data

We are using a dataset of BBC news articles for a specific month (see `get_bbc_news_sample` function below). The dataset contains the following columns: 
- `article`: The article as we would see it on the website.
- `title`: The title of the article.
- `is_business`: Whether the article is about business.
- `description`: A short summary of the article.

As you see, we start with a *labeled* data set, as is important for many use cases. If this is not available in your case, it may be worth to label a few samples to get started.

In [1]:
from llmops_training.news_reader.data import get_bbc_news_sample

In [11]:
# data = get_bbc_news_sample()
data = """
Rabobank doet een schokkende voorspelling voor de huizenmarkt in 2026

    Michiel V
    3 dagen geleden

Bijgewerkt op: 2 dagen geleden

In het kort:

    Rabobank verwacht dat huizenprijzen in 2026 en 2027 opnieuw hard stijgen door structurele krapte en tegenvallende nieuwbouw.

    De tijdelijke verkoopgolf van particuliere huurwoningen geeft kortdurend extra aanbod, maar droogt in 2026 grotendeels op.

    Door stijgende prijzen en achterblijvende lonen verslechtert de betaalbaarheid verder, wat starters en doorstromers extra onder druk zet.

Rabobank verwacht dat de Nederlandse huizenprijzen in 2026 en 2027 opnieuw fors zullen stijgen. Voor komend jaar rekent de bank op een gemiddelde prijsstijging van 4,8 procent. In 2027 komt daar naar verwachting nog eens 5,5 procent bovenop. Dat betekent dat woningen opnieuw duurder worden dan de lonen stijgen, waardoor de betaalbaarheid verder verslechtert. Volgens econoom Stefan Groot komt de totale stijging over 2025 waarschijnlijk uit op 8,8 procent en zet de opwaartse lijn dus nog minstens twee jaar door. Het zorgt ervoor dat veel huizenzoekers opnieuw tegen hogere prijzen aankijken, zelfs in een periode waarin het aanbod tijdelijk groter lijkt.
"""


data

'\nRabobank doet een schokkende voorspelling voor de huizenmarkt in 2026\n\n    Michiel V\n    3 dagen geleden\n\nBijgewerkt op: 2 dagen geleden\n\nIn het kort:\n\n    Rabobank verwacht dat huizenprijzen in 2026 en 2027 opnieuw hard stijgen door structurele krapte en tegenvallende nieuwbouw.\n\n    De tijdelijke verkoopgolf van particuliere huurwoningen geeft kortdurend extra aanbod, maar droogt in 2026 grotendeels op.\n\n    Door stijgende prijzen en achterblijvende lonen verslechtert de betaalbaarheid verder, wat starters en doorstromers extra onder druk zet.\n\nRabobank verwacht dat de Nederlandse huizenprijzen in 2026 en 2027 opnieuw fors zullen stijgen. Voor komend jaar rekent de bank op een gemiddelde prijsstijging van 4,8 procent. In 2027 komt daar naar verwachting nog eens 5,5 procent bovenop. Dat betekent dat woningen opnieuw duurder worden dan de lonen stijgen, waardoor de betaalbaarheid verder verslechtert. Volgens econoom Stefan Groot komt de totale stijging over 2025 waars

Let's print the first article which is about business:

In [12]:
article = data
print(article)


Rabobank doet een schokkende voorspelling voor de huizenmarkt in 2026

    Michiel V
    3 dagen geleden

Bijgewerkt op: 2 dagen geleden

In het kort:

    Rabobank verwacht dat huizenprijzen in 2026 en 2027 opnieuw hard stijgen door structurele krapte en tegenvallende nieuwbouw.

    De tijdelijke verkoopgolf van particuliere huurwoningen geeft kortdurend extra aanbod, maar droogt in 2026 grotendeels op.

    Door stijgende prijzen en achterblijvende lonen verslechtert de betaalbaarheid verder, wat starters en doorstromers extra onder druk zet.

Rabobank verwacht dat de Nederlandse huizenprijzen in 2026 en 2027 opnieuw fors zullen stijgen. Voor komend jaar rekent de bank op een gemiddelde prijsstijging van 4,8 procent. In 2027 komt daar naar verwachting nog eens 5,5 procent bovenop. Dat betekent dat woningen opnieuw duurder worden dan de lonen stijgen, waardoor de betaalbaarheid verder verslechtert. Volgens econoom Stefan Groot komt de totale stijging over 2025 waarschijnlijk uit op 

## Defining the LLM output

Now, let's define the Pydantic schema for the LLM to extract from each article.

> **Exercise** 📝
>
> - Fill in the TODO's below in the `ArticleInfo` Pydantic model.
> - Subsequently, fill in the prompt in the next section to extract the information from an article.
>

In [4]:
from typing import List, Literal
from pydantic import BaseModel, Field

In [5]:
class BusinessSpecificInfo(BaseModel):
    business: str = Field(..., description="The business or company involved")
    stock_price_change: Literal["increase", "decrease", "none"] = Field(
        ..., description="""
        Possible stock price change as result of the article. 
        - "increase" if article speaks positively about the business
        - "decrease" if article speaks negatively about the business
        - "none" if article speaks neutrally about the business
        """
    )
    reason: str = Field(..., description="A single sentence reason for the possible stock price change")
    relevant_substring: str = Field(
        ..., description="A relevant substring from the article supporting the reason (10-20 words)"
    )

class ArticleInfo(BaseModel):
    title: str = Field(..., description="The title of the article")
    summary: str = Field(..., description="A single sentence summary of the article")
    is_about_business: bool = Field(..., description="Whether the article is about a business")
    business_info: list[BusinessSpecificInfo] = ...  # TODO: Fill me in! This should be a List of BusinessSpecificInfo objects (no need to use Field here)

In [ ]:
# %load ../solutions/use-case-introduction/pydantic-model.py

## Extracting the information

In [6]:
from llmops_training.news_reader.generation import generate_object

In [13]:
prompt_template = "Extract structured information from the following article:\n\n{article}"

prompt = prompt_template.format(article=article)

article_info = generate_object(prompt, ArticleInfo)

article_info


ArticleInfo(title='Rabobank doet een schokkende voorspelling voor de huizenmarkt in 2026', summary='Rabobank predicts significant increases in housing prices in 2026 and 2027 due to structural shortages and limited new construction, worsening affordability.', is_about_business=True, business_info=[BusinessSpecificInfo(business='Rabobank', stock_price_change='none', reason='The bank predicts a sharp rise in housing prices based on structural shortages and insufficient new builds.', relevant_substring='Rabobank verwacht dat de Nederlandse huizenprijzen in 2026 en 2027 opnieuw fors zullen stijgen.')])

In [ ]:
# %load ../solutions/use-case-introduction/make-prompt.py

Let's see if that worked!

In [ ]:
output = generate_object(prompt, ArticleInfo)

In [ ]:
output.model_dump()

Extra: check `generate_object` and figure out how to try different models! Try some different ones and see which one works well for this task.

> Tip: Use `model_name` and try for example `gpt-5`.

## (Bonus) Deep dive Instructor

> **Exercise** 📝
>
> - Step into the underlying code from `generate_object` and try to understand how Instructor extracts structured information from the text given a Pydantic base model.
> - Does it use [Function Calling](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling)? Does it do retry iterations? Could we implement this ourselves? Discuss with your peers!

---